In [ ]:
import pyaudio
import numpy as np
import keras
import librosa
import librosa.display
import os
import glob
import scipy
import pyaudio
import numpy as np
import time
import soundfile as sf
import matplotlib.pyplot as plt

from multiprocessing import Process
from hailo_platform import (HEF, VDevice, HailoStreamInterface, InferVStreams, ConfigureParams,
    InputVStreamParams, OutputVStreamParams, InputVStreams, OutputVStreams, FormatType)

In [ ]:
windowLength = 255
fftLength = 255
hop_length = 63 
frame_length = 8064
chunk = 8064
rate = 48000

#noisy_file = 'test_audio2.wav'
noisy_file = 'noisy1.wav'
clean_file = 'teset_cleanaudio.wav'
MODEL_NAME = "VOX_MODEL"  # Replace with your model name
HEF_FILE = 'New_FFT_Vox_Model.hef'

In [ ]:
def convert_to_time_domain(predicted_clean,data_stft_phase,data_stft_mag):
    predicted_clean = np.squeeze(predicted_clean)
    print("This is the unsequeezes shaoe")
    print(predicted_clean.shape)
  
    predicted_mag_db_unscaled = (predicted_clean*80)-80
    predicted_mag = librosa.db_to_amplitude(predicted_mag_db_unscaled, ref=np.max(data_stft_mag))
    # predicted_sub = data_stft_mag - predicted_mag 
    predicted_stft = predicted_mag * data_stft_phase
    predicted_final = librosa.istft(predicted_stft ,hop_length=hop_length, length=frame_length)

    return(predicted_final)

def convert_to_stft(data):
    # data_stft = librosa.stft(data, n_fft=fftLength, win_length=windowLength, hop_length=overlap, window=window, center=True)
    data_stft = librosa.stft(data, n_fft=fftLength, hop_length=hop_length)
    data_stft_mag, data_stft_phase =librosa.magphase(data_stft)

    data_stft_mag_db = librosa.amplitude_to_db(data_stft_mag, ref=np.max)
    
    data_stft_mag_db_scaled = (data_stft_mag_db+80)/80
    print("this is the reka shape")
    print(data_stft_mag_db_scaled.shape)
    data_stft_mag_db_scaled = np.reshape(data_stft_mag_db_scaled,(1,data_stft_mag_db_scaled.shape[0],data_stft_mag_db_scaled.shape[1],1))
    return data_stft_mag_db_scaled,data_stft_mag,data_stft_phase

In [ ]:
# Chunk Audio

if __name__ == "__main__":

    # MODEL INPUT and OUTPUT
    target = VDevice()
    hef = HEF(HEF_FILE)

   
    #print(f"Output Data Type: {output_info.dtype}")
    # Configure network groups
    configure_params = ConfigureParams.create_from_hef(hef=hef, interface=HailoStreamInterface.PCIe)
    network_groups = target.configure(hef, configure_params)
    network_group = network_groups[0]
    network_group_params = network_group.create_params()

    input_vstreams_params = InputVStreamParams.make(network_group, format_type=FormatType.FLOAT32)
    output_vstreams_params = OutputVStreamParams.make(network_group, format_type=FormatType.FLOAT32)

    input_vstreams_info = hef.get_input_vstream_infos()[0]
    output_vstreams_info = hef.get_output_vstream_infos()[0]
     ##============================================================
    
    noisy_chunks = []
    clean_audio_array = []
    
    noisy_sample, noisy_sample_rate = librosa.load(noisy_file)
    print(noisy_sample.shape)
    for j in range(chunk, len(noisy_sample), chunk):
        k = j - chunk
        noisy_chunks.append(noisy_sample[k:j])
    #print(noisy_chunks)    
    noisy_chunks = np.array(noisy_chunks)
    #print(noisy_chunks)
    print(noisy_chunks[0].shape)
    
    #for i in range(len(noisy_chunks):
    for i in range(len(noisy_chunks)): 
        data_stft = librosa.stft(noisy_chunks[i], n_fft=fftLength, hop_length=hop_length)
        chunk_stft_mag, chunkstft_phase =librosa.magphase(data_stft)
        data_stft_mag_db = librosa.amplitude_to_db(chunk_stft_mag, ref=np.max)
        chunk_stft_mag_db_scaled = (data_stft_mag_db+80)/80
        print("this is the reka shape")
        print(chunk_stft_mag_db_scaled.shape)
        chunk_stft_mag_db_scaled = np.reshape(chunk_stft_mag_db_scaled,(1,chunk_stft_mag_db_scaled.shape[0],chunk_stft_mag_db_scaled.shape[1],1))
        data_stft_mag_db = np.reshape(data_stft_mag_db,(1,data_stft_mag_db.shape[0],data_stft_mag_db.shape[1],1))

        print(chunk_stft_mag_db_scaled.shape)
        
        
        #chunk_stft_mag_db_scaled, chunk_stft_mag, chunkstft_phase = convert_to_stft(noisy_chunks[i])
        print(chunk_stft_mag_db_scaled.shape)
        with InferVStreams(network_group,input_vstreams_params, output_vstreams_params) as infer_pipeline:
            input_data = {input_vstreams_info.name: data_stft_mag_db }
            with network_group.activate(network_group_params):
                    infer_results = infer_pipeline.infer(input_data)
                    #print('Stream output shape is {}'.format(infer_results[output_vstreams_info.name].shape))
        result_arr = infer_results.get('New_FFT_Vox_Model/ne_activation_conv24')
        print("this is the finla output shape")
        print(result_arr.shape)

        predicted_clean = np.squeeze(result_arr)
        print("This is the unsequeezes shape")
        print(predicted_clean.shape)
  
        predicted_mag_db_unscaled = (predicted_clean*80)-80
        librosa.display.specshow(
                predicted_mag_db_unscaled, 
                sr=rate, hop_length=hop_length, x_axis='time', y_axis='log',
                vmin=-80, vmax=0
                )
        plt.colorbar(format="%+2.0f dB")
        plt.show()
        predicted_mag = librosa.db_to_amplitude(predicted_mag_db_unscaled, ref=np.max(chunk_stft_mag))
        # predicted_sub = data_stft_mag - predicted_mag 
        predicted_stft = predicted_mag * chunkstft_phase
        clean_audio = librosa.istft(predicted_stft ,hop_length=hop_length, length=frame_length)
    
        #clean_audio = convert_to_time_domain(result_arr, chunkstft_phase, chunk_stft_mag)
        print("clean audio shape")
        print(clean_audio.shape)
        #print(chunkstft_phase.shape)
        #print(chunk_stft_mag.shape)
        clean_audio_array.append(clean_audio)
    #chunk_stft = librosa.stft(noisy_chunks[0], n_fft = fftLength, hop_length = hop_length)
    #chunk_stft_mag, chunkstft_phase = librosa.magphase(chunk_stft)
    #chunk_stft_mag_db = librosa.amplitude_to_db(chunk_stft_mag, ref = np.max)
    #chunk_stft_mag_db_scaled = (chunk_stft_mag_db + 80)/80
    #chunk_stft_mag_db_scaled = np.reshape(chunk_stft_mag_db_scaled, (1,chunk_stft_mag_db_scaled.shape[0], chunk_stft_mag_db_scaled.shape[1],1))
    #print(chunk_stft.shape)
    #print(chunk_stft_mag_db)
    #print(chunk_stft_mag_db_scaled)
    print(chunk_stft_mag_db_scaled.shape)


    #debug_wav_preprocessing(WAV_FILE)

   # streaming_inference_from_wav()
    #print(input_vstreams_info)
    #print(output_vstreams_info)
    
    #with InferVStreams(network_group,input_vstreams_params, output_vstreams_params) as infer_pipeline:
           # input_data = {input_vstreams_info.name: chunk_stft_mag_db_scaled}
           # with network_group.activate(network_group_params):
                  #  infer_results = infer_pipeline.infer(input_data)
                  #  print('Stream output shape is {}'.format(infer_results[output_vstreams_info.name].shape))

    

    